In [ ]:
# import necessary libraries
import pandas as pd
import pandas as pd
import numpy as np
import ast
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
playerMatchDetailsPerformanceDF = pd.read_csv('../dataset/aggregateMatchAndTimeline.csv')
summonerRanksDF = pd.read_csv('../dataset/summonerRanks.csv')
summonerRanksDF = summonerRanksDF[['summonerId', 'queueType', 'tier', 'rank', 'leaguePoints']] # just selecting necessary sets of data

print(playerMatchDetailsPerformanceDF.shape)
print(summonerRanksDF.shape)

for aCol in playerMatchDetailsPerformanceDF.columns:
    print(aCol)

print("-------------------------------")
for rCol in summonerRanksDF.columns:
    print(rCol)

In [ ]:
# join summonerRanks with aggregateMatchDF
playerMatchDetailsPerformanceWithRanksDF = playerMatchDetailsPerformanceDF.merge(summonerRanksDF, on='summonerId', how='inner')
print(playerMatchDetailsPerformanceWithRanksDF.shape)

# count of total players in each game; both teams; can be used for filtering matches or check sanity tests
matchPlayerCounts = playerMatchDetailsPerformanceWithRanksDF.groupby(['gameId'])['summonerId'].count().reset_index()
matchPlayerCounts.rename(columns={'summonerId': 'matchPlayerCounts'}, inplace=True)

# adds the matchPlayerCounts to the dataframe in use
playerMatchDetailsPerformanceWithRanksDF = playerMatchDetailsPerformanceWithRanksDF.merge(matchPlayerCounts, on='gameId', how='inner')
playerMatchDetailsPerformanceWithRanksDF.head()

# apply the filter of 10 players in a team to prevent cases of a single player being AFK
valid_player_match_details_df = playerMatchDetailsPerformanceWithRanksDF[playerMatchDetailsPerformanceWithRanksDF['matchPlayerCounts'] == 10]
print("Number of matches after player count filter: ", valid_player_match_details_df.shape[0] / 10) # divide by 10 because there are 10 rows for each match; corresponding to players

print("Number of unique players: ", valid_player_match_details_df['summonerId'].nunique())


In [ ]:
# of the 31055 player records (i.e. 31055 matches for 10 players in each team), the RECORDS for RANKED games should total to this number of 310550
valid_player_match_details_df['queueType'].value_counts()

## here 265853 + 44697 added to 310550. don't be confused that RANKED SOLO and RANKED FLEX always has to be multiple of 10. 
# it is possible that RANKED SOLO is being matched with opponent of RANKED FLEX 5


In [ ]:
summonerGamesPlayed = valid_player_match_details_df.groupby(['summonerId'])['gameId'].count().reset_index(name='summonerGamesPlayed')
valid_player_match_details_df = valid_player_match_details_df.merge(summonerGamesPlayed, on='summonerId', how='inner')
valid_player_match_details_df

In [ ]:
valid_player_match_details_df['summonerGamesPlayed'].describe()

In [ ]:
# copy valid_player_match_details_df to filtered_valid_player_match_details_df
filtered_valid_player_match_details_df = valid_player_match_details_df

In [ ]:
# convert player ranks to numerical ranks
# join the tier and rank to yield a combined value
filtered_valid_player_match_details_df['tierRank'] = filtered_valid_player_match_details_df['tier'] + '-' + filtered_valid_player_match_details_df['rank']

rankedDict = {
    'NA':0,
    'IRON-IV':1,
    'IRON-III':2,
    'IRON-II':3,
    'IRON-I':4,
    'BRONZE-IV':5,
    'BRONZE-III':6,
    'BRONZE-II':7,
    'BRONZE-I':8,
    'SILVER-IV':9,
    'SILVER-III':10,
    'SILVER-II':11,
    'SILVER-I':12,
    'GOLD-IV':13,
    'GOLD-III':14,
    'GOLD-II':15,
    'GOLD-I':16,
    'PLATINUM-IV':17,
    'PLATINUM-III':18,
    'PLATINUM-II':19,
    'PLATINUM-I':20,
    'EMERALD-IV':21,
    'EMERALD-III':22,
    'EMERALD-II':23,
    'EMERALD-I':24,
    'DIAMOND-IV':25,
    'DIAMOND-III':26,
    'DIAMOND-II':27,
    'DIAMOND-I':28,
    'MASTER-I':29,
    'GRANDMASTER-I':30,
    'CHALLENGER-I':31
}

# for calculating a numerical rank value for a player of a given match
def calculateNumericalRank(row):
    return rankedDict[row['tierRank']]

filtered_valid_player_match_details_df['playerNumericalRank'] = filtered_valid_player_match_details_df.apply(calculateNumericalRank, axis=1)

# averaging the rank of all 10 players in a match
matchAverageRankDF = filtered_valid_player_match_details_df.groupby(['gameId'])['playerNumericalRank'].mean().reset_index()
matchAverageRankDF.rename(columns={'playerNumericalRank': 'matchAverageRank'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(matchAverageRankDF, on='gameId', how='inner')

# averaging the rank of a given team in a match; 5 players in the team
teamAverageRankDF = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['playerNumericalRank'].mean().reset_index()
teamAverageRankDF.rename(columns={'playerNumericalRank': 'teamAverageRank'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(teamAverageRankDF, on=['gameId', 'teamId'], how='inner')

# summing up total goldEarned by a team in a match
totalTeamGold = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['goldEarned'].sum().reset_index()
totalTeamGold.rename(columns={'goldEarned': 'totalTeamGold'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamGold, on=['gameId', 'teamId'], how='inner')

# another column for total gold per minute for the team
filtered_valid_player_match_details_df['totalTeamGPM'] = np.ceil(filtered_valid_player_match_details_df['totalTeamGold'] / filtered_valid_player_match_details_df['gameDuration'])

# summing up the total kills by a team in a match
totalTeamKills = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['kills'].sum().reset_index()
totalTeamKills.rename(columns={'kills': 'totalTeamKills'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamKills, on=['gameId', 'teamId'], how='inner')

# another column for total kills per minute for the team
filtered_valid_player_match_details_df['totalTeamKPM'] = filtered_valid_player_match_details_df['totalTeamKills'] / filtered_valid_player_match_details_df['gameDuration']

# summing up the total deaths of a team in a match
totalTeamDeaths = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['deaths'].sum().reset_index()
totalTeamDeaths.rename(columns={'deaths': 'totalTeamDeaths'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamDeaths, on=['gameId', 'teamId'], how='inner')

# summing up the total assists by a team in a match
totalTeamAssists = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['assists'].sum().reset_index()
totalTeamAssists.rename(columns={'assists': 'totalTeamAssists'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamAssists, on=['gameId', 'teamId'], how='inner')

filtered_valid_player_match_details_df['totalTeamAPM'] = filtered_valid_player_match_details_df['totalTeamAssists'] / filtered_valid_player_match_details_df['gameDuration']

# summing up the total epicMonsterKills by a team in a match
totalTeamEpicMonsterKills = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['epicMonsterKills'].sum().reset_index()
totalTeamEpicMonsterKills.rename(columns={'epicMonsterKills': 'totalTeamEpicMonsterKills'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamEpicMonsterKills, on=['gameId', 'teamId'], how='inner')

# summing up the total towers taken by a team in a match
totalTeamTowerKills = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['turretKills'].sum().reset_index()
totalTeamTowerKills.rename(columns={'turretKills': 'totalTeamTurretKills'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamTowerKills, on=['gameId', 'teamId'], how='inner')

# summing up vision score for a team in a match
totalTeamVisionScore = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['visionScore'].sum().reset_index()
totalTeamVisionScore.rename(columns={'visionScore': 'totalTeamVisionScore'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamVisionScore, on=['gameId', 'teamId'], how='inner')

# another metric for vision score of the team per minute
filtered_valid_player_match_details_df['totalTeamVPM'] = filtered_valid_player_match_details_df['totalTeamVisionScore'] / filtered_valid_player_match_details_df['gameDuration']

# team average vision score in a match
averageTeamVisionScore = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['visionScore'].mean().reset_index()
averageTeamVisionScore.rename(columns={'visionScore': 'averageTeamVisionScore'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(averageTeamVisionScore, on=['gameId', 'teamId'], how='inner')

# team total champ experience in a match
totalTeamChampExperience = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['champExperience'].sum().reset_index()
totalTeamChampExperience.rename(columns={'champExperience': 'totalTeamChampExperience'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamChampExperience, on=['gameId', 'teamId'], how='inner')

# another column for total champ experience per minute per team
filtered_valid_player_match_details_df['totalTeamExpPerMin'] = filtered_valid_player_match_details_df['totalTeamChampExperience'] / filtered_valid_player_match_details_df['gameDuration']

# average champ experience per team
averageTeamChampExperience = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['champExperience'].mean().reset_index()
averageTeamChampExperience.rename(columns={'champExperience': 'averageTeamChampExperience'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(averageTeamChampExperience, on=['gameId', 'teamId'], how='inner')

# total minions killed by a team in a match
totalTeamMinionsKilled = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['totalMinionsKilled'].sum().reset_index()
totalTeamMinionsKilled.rename(columns={'totalMinionsKilled': 'totalTeamMinionsKilled'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamMinionsKilled, on=['gameId', 'teamId'], how='inner')

filtered_valid_player_match_details_df['totalTeamMinionsPerMin'] = filtered_valid_player_match_details_df['totalTeamMinionsKilled'] / filtered_valid_player_match_details_df['gameDuration']

# average minions killed per team
averageTeamMinionsKilled = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['totalMinionsKilled'].mean().reset_index()
averageTeamMinionsKilled.rename(columns={'totalMinionsKilled': 'averageTeamMinionsKilled'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(averageTeamMinionsKilled, on=['gameId', 'teamId'], how='inner')

print(filtered_valid_player_match_details_df.shape)

filtered_valid_player_match_details_df[['gameId', 'teamId', 'win', 'participantId', 'teamPosition', 'kills', 'deaths', 'epicMonsterKills','assists', 'turretKills', 'playerNumericalRank', 'teamAverageRank', 'matchAverageRank', 'totalTeamKills', 'totalTeamDeaths', 'totalTeamAssists', 'totalTeamEpicMonsterKills', 'totalTeamTurretKills']].head(20)

In [ ]:
for col in valid_player_match_details_df.columns:
    print(col)

In [ ]:
filtered_valid_player_match_details_df['visionScore'].describe()

In [ ]:
# convert the string representation of assists list into actual lists using ast
filtered_valid_player_match_details_df['participantsAssisted'] = filtered_valid_player_match_details_df['participantsAssisted'].apply(ast.literal_eval)
filtered_valid_player_match_details_df['towerKillsAssisted'] = filtered_valid_player_match_details_df['towerKillsAssisted'].apply(ast.literal_eval)
filtered_valid_player_match_details_df['monsterKillsAssisted'] = filtered_valid_player_match_details_df['monsterKillsAssisted'].apply(ast.literal_eval)
filtered_valid_player_match_details_df['participantsAssistedWithPressure'] = filtered_valid_player_match_details_df['participantsAssistedWithPressure'].apply(ast.literal_eval)

# concatenate the 4 lists into a single list
filtered_valid_player_match_details_df['totalAssisted'] = filtered_valid_player_match_details_df.apply(lambda row: [item for sublist in (row['participantsAssisted'], row['towerKillsAssisted'], row['monsterKillsAssisted'], row['participantsAssistedWithPressure']) for item in sublist], axis=1)

filtered_valid_player_match_details_df['assistsCount'] = filtered_valid_player_match_details_df['participantsAssisted'].apply(len)
filtered_valid_player_match_details_df['towerKillsAssistsCount'] = filtered_valid_player_match_details_df['towerKillsAssisted'].apply(len)
filtered_valid_player_match_details_df['monsterKillsAssistsCount'] = filtered_valid_player_match_details_df['monsterKillsAssisted'].apply(len)
filtered_valid_player_match_details_df['participantsAssistedWithPressureCount'] = filtered_valid_player_match_details_df['participantsAssistedWithPressure'].apply(len)
filtered_valid_player_match_details_df['allAssistsCount'] = filtered_valid_player_match_details_df['assistsCount'] + \
                                                            filtered_valid_player_match_details_df['towerKillsAssistsCount'] + \
                                                            filtered_valid_player_match_details_df['monsterKillsAssistsCount'] + \
                                                            filtered_valid_player_match_details_df['participantsAssistedWithPressureCount']

filtered_valid_player_match_details_df['allKills'] = filtered_valid_player_match_details_df['kills'] + filtered_valid_player_match_details_df['epicMonsterKills'] + filtered_valid_player_match_details_df['turretKills']

totalTeamAllKills = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['allKills'].sum().reset_index()
totalTeamAllKills.rename(columns={'allKills': 'totalTeamAllKills'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamAllKills, on=['gameId', 'teamId'], how='inner')

totalTeamAllAssists = filtered_valid_player_match_details_df.groupby(['gameId', 'teamId'])['allAssistsCount'].sum().reset_index()
totalTeamAllAssists.rename(columns={'allAssistsCount': 'totalTeamAllAssists'}, inplace=True)
filtered_valid_player_match_details_df = filtered_valid_player_match_details_df.merge(totalTeamAllAssists, on=['gameId', 'teamId'], how='inner')


In [ ]:
# create a sub-set of the dataframe from the valid matches that could be used to map assists
assistsMapDF = filtered_valid_player_match_details_df[[
    'gameId', 'teamId', 'win', 'teamPosition', 'teamAverageRank', 'matchAverageRank', # overall match info 
    'participantId', 'kills', 'deaths', 'assists', 'participantsAssisted', 'towerKillsAssisted', 'monsterKillsAssisted', 'participantsAssistedWithPressure', 'totalAssisted', # participant performance
    'totalTeamKills', 'totalTeamAssists', 'totalTeamTurretKills', 'totalTeamEpicMonsterKills', 'totalTeamAllKills', 'totalTeamAllAssists', # team performance
    'totalTeamGPM', 'totalTeamKPM', 'totalTeamExpPerMin', 'totalTeamVPM', 'totalTeamAPM', 'totalTeamMinionsPerMin' # per min team performance
]]


assistsMapDF.head(20)

In [ ]:
assistsMapDF['totalAssisted']

In [ ]:
## based on the totalAssisted column, create another column that holds a dictionary such that (key,value) such that 
# key = participantId that was assisted and 
# value = number of times that participant was assisted

def convertListToDict(list):
    myDict = {}
    for item in list:
        if item not in myDict.keys():
            myDict[item] = 1
        else:
            myDict[item] += 1
    return myDict

assistsMapDF['totalAssistsMap'] = assistsMapDF['totalAssisted'].apply(convertListToDict)
assistsMapDF


In [ ]:
# computing total indegree and total outdegree of individual players in the team
from collections import defaultdict

# default empty dictionary to use later for indegree assignment
indegree = defaultdict(int)

# Calculate playerOutdegree and update indegree counts; alternatively you can use it to using len of totalAssisted
assistsMapDF['playerTotalOutdegree'] = assistsMapDF['totalAssistsMap'].apply(lambda x: sum(x.values()))

for _, row in assistsMapDF.iterrows():
    for teammate, assist_count in row['totalAssistsMap'].items():
        indegree[(row['gameId'], teammate)] += assist_count

# Add playerIndegree to the DataFrame
assistsMapDF['playerTotalIndegree'] = assistsMapDF.apply(lambda row: indegree[(row['gameId'], row['participantId'])], axis=1)

In [ ]:
## computing in-degree centrality of a team
# step 1; compute max of team in-degree
teamMaxIndegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalIndegree'].max().reset_index()
teamMaxIndegree.rename(columns={'playerTotalIndegree': 'teamMaxIndegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamMaxIndegree, on=['gameId', 'teamId'], how='inner')

# step 2; compute sum of all team members in-degree
teamTotalIndegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalIndegree'].sum().reset_index()
teamTotalIndegree.rename(columns={'playerTotalIndegree': 'teamTotalIndegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamTotalIndegree, on=['gameId', 'teamId'], how='inner')

# step 3; sum up the difference between max and individual in-degree and sum it up to another column
assistsMapDF['playerDiffMaxTeamIndegree'] = assistsMapDF['teamMaxIndegree'] - assistsMapDF['playerTotalIndegree']
playerDiffMaxTeamIndegreeSum = assistsMapDF.groupby(['gameId', 'teamId'])['playerDiffMaxTeamIndegree'].sum().reset_index()
playerDiffMaxTeamIndegreeSum.rename(columns={'playerDiffMaxTeamIndegree': 'playerDiffMaxTeamIndegreeSum'}, inplace=True)
assistsMapDF = assistsMapDF.merge(playerDiffMaxTeamIndegreeSum, on=['gameId', 'teamId'], how='inner')

# step 4; the summed up column to be divided by (4 * sum of all indegrees)
assistsMapDF['teamIndegreeCentrality'] = assistsMapDF['playerDiffMaxTeamIndegreeSum'] / ( 4 * assistsMapDF['teamTotalIndegree'] )

In [ ]:
## computing out-degree centrality of a team and weight centralization of the team
# step 1; compute max of team's out-degree
teamMaxOutdegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalOutdegree'].max().reset_index()
teamMaxOutdegree.rename(columns={'playerTotalOutdegree': 'teamMaxOutdegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamMaxOutdegree, on=['gameId', 'teamId'], how='inner')

# step 2; compute sum of all team members out-degree
teamTotalOutdegree = assistsMapDF.groupby(['gameId', 'teamId'])['playerTotalOutdegree'].sum().reset_index()
teamTotalOutdegree.rename(columns={'playerTotalOutdegree': 'teamTotalOutdegree'}, inplace=True)
assistsMapDF = assistsMapDF.merge(teamTotalOutdegree, on=['gameId', 'teamId'], how='inner')

# step 3; sum up the difference between max and individual out-degree and sum it up to another column
assistsMapDF['playerDiffMaxTeamOutdegree'] = assistsMapDF['teamMaxOutdegree'] - assistsMapDF['playerTotalOutdegree']
playerDiffMaxTeamIndegreeSum = assistsMapDF.groupby(['gameId', 'teamId'])['playerDiffMaxTeamOutdegree'].sum().reset_index()
playerDiffMaxTeamIndegreeSum.rename(columns={'playerDiffMaxTeamOutdegree': 'playerDiffMaxTeamOutdegreeSum'}, inplace=True)
assistsMapDF = assistsMapDF.merge(playerDiffMaxTeamIndegreeSum, on=['gameId', 'teamId'], how='inner')

# step 4; the summed up column to be divided by (4 * sum of all indegrees)
assistsMapDF['teamOutdegreeCentrality'] = assistsMapDF['playerDiffMaxTeamOutdegreeSum'] / ( 4 * assistsMapDF['teamTotalIndegree'] ) # 4 = N-1; totalTeamIndegree is the same as totalTeamOutdegree; so you can use either of them

# for weight centralization
assistsMapDF['teamWeightCentralization'] = assistsMapDF['playerDiffMaxTeamOutdegreeSum'] / (19 * assistsMapDF['teamTotalIndegree'])

assistsMapDF

In [ ]:
# just create a dataframe with participant id and their positions
teamPositionsDF = assistsMapDF[['gameId', 'teamId', 'participantId', 'teamPosition']]
teamPositionsDF.drop_duplicates(inplace=True)
print(teamPositionsDF.shape)

In [ ]:
assistsMapDF.columns

In [ ]:
# expanding the totalAssistsMap columns to reflect the weights between nodes of players
edgesDF_rows = []

# Iterate through each row in the DataFrame
for _, row in assistsMapDF.iterrows():
    gameId = row['gameId']
    teamId = row['teamId']
    teamAverageRank = row['teamAverageRank']
    matchAverageRank = row['matchAverageRank']
    participant = row['participantId']
    totalAssistsMap = row['totalAssistsMap']
    teamIndegreeCentrality = row['teamIndegreeCentrality']
    teamOutdegreeCentrality = row['teamOutdegreeCentrality']
    teamTotalAllAssists = row['totalTeamAllAssists']
    win = row['win']
    teamGPM = row['totalTeamGPM']
    teamAverageRank = row['teamAverageRank']
    matchAverageRank = row['matchAverageRank']
    
    # Iterate through each key-value pair in the totalAssistsMap
    for to_participant, weight in totalAssistsMap.items():
        edgesDF_rows.append({
            'gameId': gameId,
            'teamId': teamId,
            'win': win,
            'fromParticipantId': participant,
            'toParticipantId': to_participant,
            'weight': weight,
            'teamIndegreeCentrality': teamIndegreeCentrality,
            'teamOutdegreeCentrality': teamOutdegreeCentrality,
            'teamTotalAllAssists': teamTotalAllAssists,
            'teamGPM': teamGPM,
            'teamAverageRank': teamAverageRank,
            'matchAverageRank': matchAverageRank
        })

# Create a new DataFrame from the expanded rows
edgesDF = pd.DataFrame(edgesDF_rows)

edgesDF['uniqueTeamId'] = edgesDF['gameId'].astype(str) + '_' + edgesDF['teamId'].astype(str)
edgesDF['normalizedWeight'] = edgesDF['weight'] / edgesDF['teamTotalAllAssists']

edgesDF.head(20)

In [ ]:
# merge team position of respective from participant id to participant id
edgesDF = edgesDF.merge(teamPositionsDF, left_on=['gameId', 'teamId', 'fromParticipantId'], right_on=['gameId', 'teamId', 'participantId'], how='inner')
edgesDF.rename(columns={'teamPosition': 'fromPlayerPosition'}, inplace=True)

edgesDF = edgesDF.merge(teamPositionsDF, left_on=['gameId', 'teamId', 'toParticipantId'], right_on=['gameId', 'teamId', 'participantId'], how='inner')
edgesDF.rename(columns={'teamPosition': 'toPlayerPosition'}, inplace=True)

In [ ]:
# computing resistance for the team by drawing a network for each team in each match and calculate resistance
def CalcResistance(eigenvalues,N=5):
    eigval = eigenvalues.tolist()
    roundedEigval = [round(val, 2) for val in eigval]
    roundedEigval = [i for i in roundedEigval if i != 0]

    summ = 0
    for e in roundedEigval:
        summ = summ + (1/e)
    R = 5 * summ
    return R

# for calculating graph resistance of teams using the edgesDF
uniqueTeamIdList = edgesDF['uniqueTeamId'].tolist()
uniqueTeamIdList = list(set(uniqueTeamIdList))
resistanceDict = {}

for idx, uid in enumerate(uniqueTeamIdList):
    uniqueTeamDF = edgesDF[edgesDF['uniqueTeamId'] == uid]
    
    # create a network graph object using network library; compute a
    G = nx.from_pandas_edgelist(uniqueTeamDF,'fromParticipantId','toParticipantId', edge_attr='weight', create_using=nx.MultiGraph(directed=True))
    A = nx.adjacency_matrix(G) # adjacency matrix
    D = np.diag(np.sum(np.array(A.todense()), axis=1)) # degree matrix
    L = D - A # laplacian matrix
    e, v = np.linalg.eig(L) # eigen values

    resistanceValue = CalcResistance(e)
    resistanceDict[uid] = round(resistanceValue,4)

teamResistance = pd.DataFrame(resistanceDict.items(), columns=['uniqueTeamId', 'resistance'])

# merge teamResistance DF into edgesDF using uniqueTeamId key
edgesDF = edgesDF.merge(teamResistance, on='uniqueTeamId', how='inner')
print(edgesDF.shape)

# merge teamResistance to assistsMapDF
assistsMapDF['uniqueTeamId'] = assistsMapDF['gameId'].astype(str) + '_' + assistsMapDF['teamId'].astype(str)
assistsMapDF = assistsMapDF.merge(teamResistance, on='uniqueTeamId', how='inner')
assistsMapDF.head(10)


In [ ]:
# read team_factor_and_labels.csv into a DF
team_labels_df = pd.read_csv('../dataset/team_factor_and_labels.csv')
individual_labels_df = pd.read_csv('../dataset/individual_factor_and_labels.csv')

In [ ]:
team_labels_df

In [ ]:
# fetch team labels column
teamLabels = team_labels_df[['gameId', 'teamId', 'teamClusterLabel']]
print(teamLabels.head())

teamLabels.shape

In [ ]:
individual_labels_df

In [ ]:
# fetch player labels columns
playerLabels = individual_labels_df[['gameId', 'teamId', 'teamPosition', 'playerClusterLabel']]
print(playerLabels.head())

playerLabels.shape

In [ ]:
# merge playerLabels and teamLabels df 
labels_df = playerLabels.merge(teamLabels, on=['gameId', 'teamId'], how='inner')
print(labels_df.shape)
labels_df.head()

In [ ]:
# create a column to count the number of good players in a team
goodPlayersCount = labels_df[labels_df['playerClusterLabel'] == 'good'].groupby(['gameId', 'teamId'])['teamPosition'].count().reset_index(name='goodPlayersCount')
labels_df = labels_df.merge(goodPlayersCount, on=['gameId', 'teamId'], how='left')
labels_df['goodPlayersCount'].fillna(0, inplace=True)

In [ ]:
labels_df['majorityGoodPlayers'] = np.where(labels_df['goodPlayersCount'] >= 3, True, False)
labels_df

In [ ]:
# bad players count in a team
badPlayersCount = labels_df[labels_df['playerClusterLabel'] == 'bad'].groupby(['gameId', 'teamId'])['teamPosition'].count().reset_index(name='badPlayersCount')
labels_df = labels_df.merge(badPlayersCount, on=['gameId', 'teamId'], how='left')
labels_df['badPlayersCount'].fillna(0, inplace=True)

labels_df['majorityBadPlayers'] = np.where(labels_df['badPlayersCount'] >= 3, True, False)
labels_df

In [ ]:
def computeTeamMajorityType(row):
    if (row['majorityGoodPlayers'] == True) and (row['teamClusterLabel'] == 'good'):
        return 'GTGP'
    elif (row['majorityGoodPlayers'] == False) and (row['teamClusterLabel'] == 'good'):
        return 'GTBP'
    elif (row['majorityGoodPlayers'] == True) and (row['teamClusterLabel'] == 'bad'):
        return 'BTGP'
    elif (row['majorityGoodPlayers'] == False) and (row['teamClusterLabel'] == 'bad'):
        return 'BTBP'
    
labels_df['teamType'] = labels_df.apply(computeTeamMajorityType, axis=1)
labels_df

In [ ]:
labels_df['teamType'].value_counts()

In [ ]:
labels_df['teamType'].isna().sum()

#### Sanity Check: The sum of teamType value counts with all majority labels assigned and the teamType with no majority labels are set as NA, and their sum totals to 310,550. This equals to the total number of players in the dataset (31055 X 10) i.e. 125660 + 90395 + 52190 + 6830 + 35475 = 310550

In [ ]:
gtgp_game_ids = labels_df[labels_df['teamType'] == 'GTGP'][['gameId', 'teamType', 'teamId']]
gtgp_game_ids.drop_duplicates(inplace=True)
print(gtgp_game_ids.shape)

gtbp_game_ids = labels_df[labels_df['teamType'] == 'GTBP'][['gameId', 'teamType', 'teamId']]
gtbp_game_ids.drop_duplicates(inplace=True)
print(gtbp_game_ids.shape)

btgp_game_ids = labels_df[labels_df['teamType'] == 'BTGP'][['gameId', 'teamType', 'teamId']]
btgp_game_ids.drop_duplicates(inplace=True)
print(btgp_game_ids.shape)

btbp_game_ids = labels_df[labels_df['teamType'] == 'BTBP'][['gameId', 'teamType', 'teamId']]
btbp_game_ids.drop_duplicates(inplace=True)
print(btbp_game_ids.shape)


In [ ]:
def convertNumericalRank(nRank):
    if nRank > 30:
        return 'CHALLENGER'
    elif nRank <=30 and nRank > 29:
        return 'GRANDMASTER'
    elif nRank <= 29 and nRank > 28:
        return 'MASTER'
    elif nRank <= 28 and nRank > 24:
        return 'DIAMOND'
    elif nRank <= 24 and nRank > 20:
        return 'EMERALD'
    elif nRank <= 20 and nRank > 16:
        return 'PLATINUM'
    elif nRank <= 16 and nRank > 12:
        return 'GOLD'
    elif nRank <= 12 and nRank > 8:
        return 'SILVER'
    elif nRank <= 8 and nRank > 4:
        return 'BRONZE'
    else:
        return 'IRON'
    
edgesDF['gameRank'] = edgesDF['matchAverageRank'].apply(convertNumericalRank)
edgesDF['teamRank'] = edgesDF['teamAverageRank'].apply(convertNumericalRank)

print(edgesDF.columns)

### Plot for good team with majority of good players

In [ ]:
selectiveGameIds = [gameId for gameId in gtgp_game_ids['gameId'].values]
print(len(selectiveGameIds))
selectiveTeamIds = [teamId for teamId in gtgp_game_ids['teamId'].values]
print(len(selectiveTeamIds))

folderPath = '../graphs/GoodTeamGoodPlayers'

# use incremental ranges of gameIds to save figure locally
for idx, (teamId, gameId) in enumerate(zip(selectiveTeamIds[0:2000], selectiveGameIds[0:2000])):
    # team level data of two teams in the same match-up
    fig, axes = plt.subplots(figsize=(6, 6))
    if teamId == 100:
        blueTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 100)]
        blueTeamDF = blueTeamDF[(blueTeamDF['toParticipantId'] >= 1) & (blueTeamDF['toParticipantId'] <= 5)] # additional filter to remove any steals or lease
        blueTeamEdges = [ (i*40) for i in blueTeamDF['normalizedWeight'] ]

        blueTeamAvgRank = blueTeamDF['teamRank'].iloc[0]
        blueTeamWins = blueTeamDF['win'].iloc[0]
        blueTeamGPM = blueTeamDF['teamGPM'].iloc[0]
        blueTeamInD = blueTeamDF['teamIndegreeCentrality'].iloc[0]
        blueTeamOutD = blueTeamDF['teamOutdegreeCentrality'].iloc[0]
        blueTeamRes = blueTeamDF['resistance'].iloc[0]
    
        blueTeamText = f'WIN: {blueTeamWins} | Team Rank: {blueTeamAvgRank} | Gold Per Min.: {blueTeamGPM} \n In-Degree: {blueTeamInD:.3f} | Out-Degree: {blueTeamOutD:.3f} | Resistance: {blueTeamRes:.3f}'
        matchRank = blueTeamDF['gameRank'].iloc[0]
        titleText = f'Match ID: {gameId}    Match Tier: {matchRank}'

        # create network graph objects
        BTG = nx.from_pandas_edgelist(blueTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))
        posB = nx.circular_layout(BTG)

        nx.draw(BTG, pos=posB, ax=axes, with_labels=True, node_size=2100, node_color="lightblue", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=blueTeamEdges, edge_color='gray')
        axes.set_title(blueTeamText, fontsize=14)
    
    else:
        redTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 200)]
        redTeamDF = redTeamDF[(redTeamDF['toParticipantId'] >= 6) & (redTeamDF['toParticipantId'] <= 10)] # additional filter to remove any steal or lease
        redTeamEdges = [ (i*40) for i in redTeamDF['normalizedWeight'] ]

        redTeamAvgRank = redTeamDF['teamRank'].iloc[0]
        redTeamWins = redTeamDF['win'].iloc[0]
        redTeamGPM = redTeamDF['teamGPM'].iloc[0]
        redTeamGPM = redTeamDF['teamGPM'].iloc[0]
        redTeamInD = redTeamDF['teamIndegreeCentrality'].iloc[0]
        redTeamOutD = redTeamDF['teamOutdegreeCentrality'].iloc[0]
        redTeamRes = redTeamDF['resistance'].iloc[0]

        redTeamText = f'WIN: {redTeamWins} | Team Rank: {redTeamAvgRank} | Gold Per Min.: {redTeamGPM} \n In-Degree: {redTeamInD:.3f} | Out-Degree: {redTeamOutD:.3f} | Resistance: {redTeamRes:.3f}'
        matchRank = redTeamDF['gameRank'].iloc[0]
        titleText = f'Match ID: {gameId}    Match Tier: {matchRank}'
        
        # create network graph objects
        RTG = nx.from_pandas_edgelist(redTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))
        posR = nx.circular_layout(RTG)

        nx.draw(RTG, pos=posR, ax=axes, with_labels=True, node_size=2100, node_color="red", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=redTeamEdges, edge_color='gray')
        axes.set_title(redTeamText, fontsize=14)

    plt.tight_layout()
    # plt.show()

    # save the figure
    output_path = f'{folderPath}/{gameId}_{teamId}.png'
    plt.savefig(output_path, format='png', dpi=300)
    print(f"Image {output_path} saved successfully")

    # clear and close the figures
    plt.clf()
    plt.close()


### Plot for good teams with majority of bad players

In [ ]:
selectiveGameIds = [gameId for gameId in gtbp_game_ids['gameId'].values]
print(len(selectiveGameIds))
selectiveTeamIds = [teamId for teamId in gtbp_game_ids['teamId'].values]
print(len(selectiveTeamIds))

folderPath = '../graphs/GoodTeamBadPlayers'

# use incremental ranges of gameIds to save figure locally
for idx, (teamId, gameId) in enumerate(zip(selectiveTeamIds[0:2000], selectiveGameIds[0:2000])):
    # team level data of two teams in the same match-up
    fig, axes = plt.subplots(figsize=(6, 6))
    if teamId == 100:
        blueTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 100)]
        blueTeamDF = blueTeamDF[(blueTeamDF['toParticipantId'] >= 1) & (blueTeamDF['toParticipantId'] <= 5)] # additional filter to remove any steals or lease
        blueTeamEdges = [ (i*40) for i in blueTeamDF['normalizedWeight'] ]

        blueTeamAvgRank = blueTeamDF['teamRank'].iloc[0]
        blueTeamWins = blueTeamDF['win'].iloc[0]
        blueTeamGPM = blueTeamDF['teamGPM'].iloc[0]
        blueTeamInD = blueTeamDF['teamIndegreeCentrality'].iloc[0]
        blueTeamOutD = blueTeamDF['teamOutdegreeCentrality'].iloc[0]
        blueTeamRes = blueTeamDF['resistance'].iloc[0]
    
        blueTeamText = f'WIN: {blueTeamWins} | Team Rank: {blueTeamAvgRank} | Gold Per Min.: {blueTeamGPM} \n In-Degree: {blueTeamInD:.3f} | Out-Degree: {blueTeamOutD:.3f} | Resistance: {blueTeamRes:.3f}'
        matchRank = blueTeamDF['gameRank'].iloc[0]
        titleText = f'Match ID: {gameId}    Match Tier: {matchRank}'

        # create network graph objects
        BTG = nx.from_pandas_edgelist(blueTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))
        posB = nx.circular_layout(BTG)

        nx.draw(BTG, pos=posB, ax=axes, with_labels=True, node_size=2100, node_color="lightblue", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=blueTeamEdges, edge_color='gray')
        axes.set_title(blueTeamText, fontsize=14)
    
    else:
        redTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 200)]
        redTeamDF = redTeamDF[(redTeamDF['toParticipantId'] >= 6) & (redTeamDF['toParticipantId'] <= 10)] # additional filter to remove any steal or lease
        redTeamEdges = [ (i*40) for i in redTeamDF['normalizedWeight'] ]

        redTeamAvgRank = redTeamDF['teamRank'].iloc[0]
        redTeamWins = redTeamDF['win'].iloc[0]
        redTeamGPM = redTeamDF['teamGPM'].iloc[0]
        redTeamGPM = redTeamDF['teamGPM'].iloc[0]
        redTeamInD = redTeamDF['teamIndegreeCentrality'].iloc[0]
        redTeamOutD = redTeamDF['teamOutdegreeCentrality'].iloc[0]
        redTeamRes = redTeamDF['resistance'].iloc[0]

        redTeamText = f'WIN: {redTeamWins} | Team Rank: {redTeamAvgRank} | Gold Per Min.: {redTeamGPM} \n In-Degree: {redTeamInD:.3f} | Out-Degree: {redTeamOutD:.3f} | Resistance: {redTeamRes:.3f}'
        matchRank = redTeamDF['gameRank'].iloc[0]
        titleText = f'Match ID: {gameId}    Match Tier: {matchRank}'
        
        # create network graph objects
        RTG = nx.from_pandas_edgelist(redTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))
        posR = nx.circular_layout(RTG)

        nx.draw(RTG, pos=posR, ax=axes, with_labels=True, node_size=2100, node_color="red", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=redTeamEdges, edge_color='gray')
        axes.set_title(redTeamText, fontsize=14)

    plt.tight_layout()
    # plt.show()

    # save the figure
    output_path = f'{folderPath}/{gameId}_{teamId}.png'
    plt.savefig(output_path, format='png', dpi=300)
    print(f"Image {output_path} saved successfully")

    # clear and close the figures
    plt.clf()
    plt.close()


### Plot for bad team with majority of good players

In [ ]:
selectiveGameIds = [gameId for gameId in btgp_game_ids['gameId'].values]
print(len(selectiveGameIds))
selectiveTeamIds = [teamId for teamId in btgp_game_ids['teamId'].values]
print(len(selectiveTeamIds))

folderPath = '../graphs/BadTeamGoodPlayers'

# use incremental ranges of gameIds to save figure locally
for idx, (teamId, gameId) in enumerate(zip(selectiveTeamIds[0:2000], selectiveGameIds[0:2000])):
    # team level data of two teams in the same match-up
    fig, axes = plt.subplots(figsize=(6, 6))
    if teamId == 100:
        blueTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 100)]
        blueTeamDF = blueTeamDF[(blueTeamDF['toParticipantId'] >= 1) & (blueTeamDF['toParticipantId'] <= 5)] # additional filter to remove any steals or lease
        blueTeamEdges = [ (i*40) for i in blueTeamDF['normalizedWeight'] ]

        blueTeamAvgRank = blueTeamDF['teamRank'].iloc[0]
        blueTeamWins = blueTeamDF['win'].iloc[0]
        blueTeamGPM = blueTeamDF['teamGPM'].iloc[0]
        blueTeamInD = blueTeamDF['teamIndegreeCentrality'].iloc[0]
        blueTeamOutD = blueTeamDF['teamOutdegreeCentrality'].iloc[0]
        blueTeamRes = blueTeamDF['resistance'].iloc[0]
    
        blueTeamText = f'WIN: {blueTeamWins} | Team Rank: {blueTeamAvgRank} | Gold Per Min.: {blueTeamGPM} \n In-Degree: {blueTeamInD:.3f} | Out-Degree: {blueTeamOutD:.3f} | Resistance: {blueTeamRes:.3f}'
        matchRank = blueTeamDF['gameRank'].iloc[0]
        titleText = f'Match ID: {gameId}    Match Tier: {matchRank}'

        # create network graph objects
        BTG = nx.from_pandas_edgelist(blueTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))
        posB = nx.circular_layout(BTG)

        nx.draw(BTG, pos=posB, ax=axes, with_labels=True, node_size=2100, node_color="lightblue", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=blueTeamEdges, edge_color='gray')
        axes.set_title(blueTeamText, fontsize=14)
    
    else:
        redTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 200)]
        redTeamDF = redTeamDF[(redTeamDF['toParticipantId'] >= 6) & (redTeamDF['toParticipantId'] <= 10)] # additional filter to remove any steal or lease
        redTeamEdges = [ (i*40) for i in redTeamDF['normalizedWeight'] ]

        redTeamAvgRank = redTeamDF['teamRank'].iloc[0]
        redTeamWins = redTeamDF['win'].iloc[0]
        redTeamGPM = redTeamDF['teamGPM'].iloc[0]
        redTeamGPM = redTeamDF['teamGPM'].iloc[0]
        redTeamInD = redTeamDF['teamIndegreeCentrality'].iloc[0]
        redTeamOutD = redTeamDF['teamOutdegreeCentrality'].iloc[0]
        redTeamRes = redTeamDF['resistance'].iloc[0]

        redTeamText = f'WIN: {redTeamWins} | Team Rank: {redTeamAvgRank} | Gold Per Min.: {redTeamGPM} \n In-Degree: {redTeamInD:.3f} | Out-Degree: {redTeamOutD:.3f} | Resistance: {redTeamRes:.3f}'
        matchRank = redTeamDF['gameRank'].iloc[0]
        titleText = f'Match ID: {gameId}    Match Tier: {matchRank}'
        
        # create network graph objects
        RTG = nx.from_pandas_edgelist(redTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))
        posR = nx.circular_layout(RTG)

        nx.draw(RTG, pos=posR, ax=axes, with_labels=True, node_size=2100, node_color="red", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=redTeamEdges, edge_color='gray')
        axes.set_title(redTeamText, fontsize=14)

    plt.tight_layout()
    # plt.show()

    # save the figure
    output_path = f'{folderPath}/{gameId}_{teamId}.png'
    plt.savefig(output_path, format='png', dpi=300)
    print(f"Image {output_path} saved successfully")

    # clear and close the figures
    plt.clf()
    plt.close()


### Plot for bad team with majority of bad players

In [ ]:
selectiveGameIds = [gameId for gameId in btbp_game_ids['gameId'].values]
print(len(selectiveGameIds))
selectiveTeamIds = [teamId for teamId in btbp_game_ids['teamId'].values]
print(len(selectiveTeamIds))

folderPath = '../graphs/BadTeamBadPlayers'

# use incremental ranges of gameIds to save figure locally
for idx, (teamId, gameId) in enumerate(zip(selectiveTeamIds[0:2000], selectiveGameIds[0:2000])):
    # team level data of two teams in the same match-up
    fig, axes = plt.subplots(figsize=(6, 6))
    if teamId == 100:
        blueTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 100)]
        blueTeamDF = blueTeamDF[(blueTeamDF['toParticipantId'] >= 1) & (blueTeamDF['toParticipantId'] <= 5)] # additional filter to remove any steals or lease
        blueTeamEdges = [ (i*40) for i in blueTeamDF['normalizedWeight'] ]

        blueTeamAvgRank = blueTeamDF['teamRank'].iloc[0]
        blueTeamWins = blueTeamDF['win'].iloc[0]
        blueTeamGPM = blueTeamDF['teamGPM'].iloc[0]
        blueTeamInD = blueTeamDF['teamIndegreeCentrality'].iloc[0]
        blueTeamOutD = blueTeamDF['teamOutdegreeCentrality'].iloc[0]
        blueTeamRes = blueTeamDF['resistance'].iloc[0]
    
        blueTeamText = f'WIN: {blueTeamWins} | Team Rank: {blueTeamAvgRank} | Gold Per Min.: {blueTeamGPM} \n In-Degree: {blueTeamInD:.3f} | Out-Degree: {blueTeamOutD:.3f} | Resistance: {blueTeamRes:.3f}'
        matchRank = blueTeamDF['gameRank'].iloc[0]
        titleText = f'Match ID: {gameId}    Match Tier: {matchRank}'

        # create network graph objects
        BTG = nx.from_pandas_edgelist(blueTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))
        posB = nx.circular_layout(BTG)

        nx.draw(BTG, pos=posB, ax=axes, with_labels=True, node_size=2100, node_color="lightblue", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=blueTeamEdges, edge_color='gray')
        axes.set_title(blueTeamText, fontsize=14)
    
    else:
        redTeamDF = edgesDF[(edgesDF['gameId'] == gameId) & (edgesDF['teamId'] == 200)]
        redTeamDF = redTeamDF[(redTeamDF['toParticipantId'] >= 6) & (redTeamDF['toParticipantId'] <= 10)] # additional filter to remove any steal or lease
        redTeamEdges = [ (i*40) for i in redTeamDF['normalizedWeight'] ]

        redTeamAvgRank = redTeamDF['teamRank'].iloc[0]
        redTeamWins = redTeamDF['win'].iloc[0]
        redTeamGPM = redTeamDF['teamGPM'].iloc[0]
        redTeamGPM = redTeamDF['teamGPM'].iloc[0]
        redTeamInD = redTeamDF['teamIndegreeCentrality'].iloc[0]
        redTeamOutD = redTeamDF['teamOutdegreeCentrality'].iloc[0]
        redTeamRes = redTeamDF['resistance'].iloc[0]

        redTeamText = f'WIN: {redTeamWins} | Team Rank: {redTeamAvgRank} | Gold Per Min.: {redTeamGPM} \n In-Degree: {redTeamInD:.3f} | Out-Degree: {redTeamOutD:.3f} | Resistance: {redTeamRes:.3f}'
        matchRank = redTeamDF['gameRank'].iloc[0]
        titleText = f'Match ID: {gameId}    Match Tier: {matchRank}'
        
        # create network graph objects
        RTG = nx.from_pandas_edgelist(redTeamDF, source='fromPlayerPosition', target='toPlayerPosition', edge_attr='normalizedWeight', create_using=nx.MultiDiGraph(directed=True))
        posR = nx.circular_layout(RTG)

        nx.draw(RTG, pos=posR, ax=axes, with_labels=True, node_size=2100, node_color="red", font_size=8, connectionstyle='arc3,rad=0.1', font_weight="bold", arrowsize=8, width=redTeamEdges, edge_color='gray')
        axes.set_title(redTeamText, fontsize=14)

    plt.tight_layout()
    # plt.show()

    # save the figure
    output_path = f'{folderPath}/{gameId}_{teamId}.png'
    plt.savefig(output_path, format='png', dpi=300)
    print(f"Image {output_path} saved successfully")

    # clear and close the figures
    plt.clf()
    plt.close()
